In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./cse151b_data/train.csv")
print(df.head)

<bound method NDFrame.head of                      TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0        1372636858620000589         C          NaN           NaN  20000589   
1        1372637303620000596         B          NaN           7.0  20000596   
2        1372636951620000320         C          NaN           NaN  20000320   
3        1372636854620000520         C          NaN           NaN  20000520   
4        1372637091620000337         C          NaN           NaN  20000337   
...                      ...       ...          ...           ...       ...   
1710665  1404171463620000698         C          NaN           NaN  20000698   
1710666  1404171367620000670         C          NaN           NaN  20000670   
1710667  1388745716620000264         C          NaN           NaN  20000264   
1710668  1404141826620000248         B          NaN          12.0  20000248   
1710669  1404157147620000079         B          NaN          34.0  20000079   

          TIMESTAMP D

In [3]:
type_a = df[df['CALL_TYPE'] == 'A']
print(len(type_a[type_a['POLYLINE'] == '[]']))
print(len(type_a))

425
364770


In [4]:
print(df.dtypes)

TRIP_ID           int64
CALL_TYPE        object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
MISSING_DATA       bool
POLYLINE         object
dtype: object


In [5]:
# drop any obsevations with missing values
df = df[df['MISSING_DATA'] == False]
print(df.shape)
no_polyline = df[df['POLYLINE'] == '[]']
print(len(no_polyline))
print(len(no_polyline[no_polyline['DAY_TYPE'] == 'A']))

(1710660, 9)
5901
5901


In [6]:
test_row = df['POLYLINE'][0].strip("[]")
print(test_row)
test_row = "[" + test_row + "]"
print(df['POLYLINE'][0])
print(test_row)
for i in range(len(test_row) - 1):
    if test_row[i-1] == ']' and test_row[i] == ',' and test_row[i+1] == '[':
        test_row = test_row[:i] + '#' + test_row [i + 1:]

test_row = test_row.split("#")

characters = ['[', ']']
longitude = []
latitude = []
for i in range(len(test_row)):
    temp = test_row[i]
    for j in range(len(characters)):
        temp = temp.replace(characters[j], '')
        temp_list = temp.split(',')
    longitude.append(temp_list[0])
    latitude.append(temp_list[1])

print(len(longitude) == len(latitude))

-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489
[[-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489]]
[-

In [7]:
def separate_polyline(coordinates):
    if coordinates == '[]':
        return [], []
    test_row = coordinates.strip("[]")
    test_row = "[" + test_row + "]"

    for i in range(len(test_row) - 1):
        if test_row[i-1] == ']' and test_row[i] == ',' and test_row[i+1] == '[':
            test_row = test_row[:i] + '#' + test_row [i + 1:]

    test_row = test_row.split("#")

    longitude = []
    latitude = []
    for i in range(len(test_row)):
        temp = test_row[i]
        temp.strip('[]')
        
        temp_list = temp.split(',')

        longitude.append(float(temp_list[0]))
        latitude.append(float(temp_list[1]))
        
    return longitude, latitude

In [8]:
df['longitude'], df['latitude'] = zip(*df['POLYLINE'].apply(separate_polyline))
print(df.columns)

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE', 'longitude',
       'latitude'],
      dtype='object')


In [9]:
df = df.drop(['POLYLINE'], axis=1)
print(df.columns)

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'longitude', 'latitude'],
      dtype='object')


In [10]:
print(df.dtypes)
print(len(df[df['MISSING_DATA'] == True]))

df = df.drop(['MISSING_DATA'], axis=1)
print(df.columns)

TRIP_ID           int64
CALL_TYPE        object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
MISSING_DATA       bool
longitude        object
latitude         object
dtype: object
0
Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'longitude', 'latitude'],
      dtype='object')


In [11]:
# Calculates travel time in seconds
def calculate_travel_time(coordinates):
    total = 0
    checks = len(coordinates) - 1
    return total + (checks * 15)

In [12]:
df['travel_time'] = df['longitude'].apply(calculate_travel_time)
df = df.drop(['longitude', 'latitude', 'CALL_TYPE', 'TRIP_ID', 'ORIGIN_CALL'], axis=1)
print(df.dtypes)

ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
travel_time       int64
dtype: object


In [13]:
def convert_day_type(day):
    if day == 'A':
        return 1
    elif day == 'B':
        return 2
    else:
        return 3

In [14]:
df['DAY_TYPE'] = df['DAY_TYPE'].apply(convert_day_type)
df['DAY_TYPE'].head()

0    1
1    1
2    1
3    1
4    1
Name: DAY_TYPE, dtype: int64

In [15]:
# use df.to_pickle() in order to refer to df much faster

df.to_pickle("train_cleaned.pkl")